In [1]:
import torch
from torch import nn
torch.manual_seed(2099)

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))

In [3]:
X = torch.rand(size=(2, 4))

In [4]:
net(X)

tensor([[-0.1282],
        [-0.0987]], grad_fn=<AddmmBackward0>)

In [5]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

#### 1. 访问参数

In [6]:
net[0].state_dict()

OrderedDict([('weight',
              tensor([[-0.1877,  0.4050, -0.4963,  0.3841],
                      [ 0.3765,  0.3211, -0.3570,  0.4775],
                      [-0.3840, -0.4015, -0.0134, -0.0548],
                      [-0.4021,  0.0073, -0.0681,  0.1147],
                      [-0.0604,  0.2029,  0.4182,  0.2056],
                      [-0.0390,  0.3900, -0.1084, -0.1969],
                      [-0.4476, -0.3049, -0.0990,  0.4068],
                      [ 0.0343,  0.4048, -0.2852,  0.2114]])),
             ('bias',
              tensor([-0.3736, -0.1743, -0.4957,  0.0392,  0.4622, -0.2760, -0.0108, -0.3508]))])

In [7]:
net[1], net[1].state_dict()

(ReLU(), OrderedDict())

In [8]:
net[2], net[2].state_dict()

(Linear(in_features=8, out_features=1, bias=True),
 OrderedDict([('weight',
               tensor([[-0.1374, -0.0657,  0.3024, -0.0057,  0.2794,  0.1247,  0.0791,  0.1464]])),
              ('bias', tensor([-0.2930]))]))

In [9]:
print(type(net[2].bias))

<class 'torch.nn.parameter.Parameter'>


In [10]:
print(net[2].bias)

Parameter containing:
tensor([-0.2930], requires_grad=True)


In [11]:
print(net[2].bias.data)

tensor([-0.2930])


In [12]:
net[2].weight.grad is None

True

#### 一次性访问所有参数

In [17]:
print(*[(name, para) for name, para in net[2].named_parameters()])

('weight', Parameter containing:
tensor([[-0.1374, -0.0657,  0.3024, -0.0057,  0.2794,  0.1247,  0.0791,  0.1464]],
       requires_grad=True)) ('bias', Parameter containing:
tensor([-0.2930], requires_grad=True))


In [18]:
print(*[(name, para) for name, para in net.named_parameters()])

('0.weight', Parameter containing:
tensor([[-0.1877,  0.4050, -0.4963,  0.3841],
        [ 0.3765,  0.3211, -0.3570,  0.4775],
        [-0.3840, -0.4015, -0.0134, -0.0548],
        [-0.4021,  0.0073, -0.0681,  0.1147],
        [-0.0604,  0.2029,  0.4182,  0.2056],
        [-0.0390,  0.3900, -0.1084, -0.1969],
        [-0.4476, -0.3049, -0.0990,  0.4068],
        [ 0.0343,  0.4048, -0.2852,  0.2114]], requires_grad=True)) ('0.bias', Parameter containing:
tensor([-0.3736, -0.1743, -0.4957,  0.0392,  0.4622, -0.2760, -0.0108, -0.3508],
       requires_grad=True)) ('2.weight', Parameter containing:
tensor([[-0.1374, -0.0657,  0.3024, -0.0057,  0.2794,  0.1247,  0.0791,  0.1464]],
       requires_grad=True)) ('2.bias', Parameter containing:
tensor([-0.2930], requires_grad=True))


In [21]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.1877,  0.4050, -0.4963,  0.3841],
                      [ 0.3765,  0.3211, -0.3570,  0.4775],
                      [-0.3840, -0.4015, -0.0134, -0.0548],
                      [-0.4021,  0.0073, -0.0681,  0.1147],
                      [-0.0604,  0.2029,  0.4182,  0.2056],
                      [-0.0390,  0.3900, -0.1084, -0.1969],
                      [-0.4476, -0.3049, -0.0990,  0.4068],
                      [ 0.0343,  0.4048, -0.2852,  0.2114]])),
             ('0.bias',
              tensor([-0.3736, -0.1743, -0.4957,  0.0392,  0.4622, -0.2760, -0.0108, -0.3508])),
             ('2.weight',
              tensor([[-0.1374, -0.0657,  0.3024, -0.0057,  0.2794,  0.1247,  0.0791,  0.1464]])),
             ('2.bias', tensor([-0.2930]))])

In [22]:
net.state_dict()['2.weight']

tensor([[-0.1374, -0.0657,  0.3024, -0.0057,  0.2794,  0.1247,  0.0791,  0.1464]])

In [23]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))

In [24]:
rgnet(X)

tensor([[-0.2949],
        [-0.2949]], grad_fn=<AddmmBackward0>)

In [25]:
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

In [29]:
rgnet[0][1][2].bias.data

tensor([-0.0416,  0.2661, -0.0609, -0.2617])

#### 2. 参数初始化

In [30]:
def init_normal(m):
    if type(m) == nn.Linear:
        # 内置参数初始化器
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

In [31]:
net.apply(init_normal)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [32]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[ 0.0054, -0.0159, -0.0021, -0.0167],
                      [ 0.0096, -0.0146,  0.0041,  0.0055],
                      [ 0.0150, -0.0083,  0.0065,  0.0123],
                      [ 0.0033,  0.0040, -0.0011, -0.0125],
                      [ 0.0030, -0.0108, -0.0172,  0.0113],
                      [ 0.0110, -0.0055,  0.0042,  0.0040],
                      [ 0.0014,  0.0029, -0.0013,  0.0087],
                      [-0.0105,  0.0114,  0.0002,  0.0215]])),
             ('0.bias', tensor([0., 0., 0., 0., 0., 0., 0., 0.])),
             ('2.weight',
              tensor([[-0.0187, -0.0077, -0.0074,  0.0103,  0.0027,  0.0017,  0.0057,  0.0026]])),
             ('2.bias', tensor([0.]))])

In [33]:
def init_constant(m):
    if type(m) == nn.Linear:
        # 参数初始化为常数
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

In [34]:
net.apply(init_constant)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [35]:
net[2].state_dict()

OrderedDict([('weight', tensor([[1., 1., 1., 1., 1., 1., 1., 1.]])),
             ('bias', tensor([0.]))])

In [36]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_2099(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 2.099)

In [37]:
net[0].apply(init_2099)
net[2].apply(init_xavier)

Linear(in_features=8, out_features=1, bias=True)

In [38]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[2.0990, 2.0990, 2.0990, 2.0990],
                      [2.0990, 2.0990, 2.0990, 2.0990],
                      [2.0990, 2.0990, 2.0990, 2.0990],
                      [2.0990, 2.0990, 2.0990, 2.0990],
                      [2.0990, 2.0990, 2.0990, 2.0990],
                      [2.0990, 2.0990, 2.0990, 2.0990],
                      [2.0990, 2.0990, 2.0990, 2.0990],
                      [2.0990, 2.0990, 2.0990, 2.0990]])),
             ('0.bias', tensor([0., 0., 0., 0., 0., 0., 0., 0.])),
             ('2.weight',
              tensor([[ 0.5799,  0.1993,  0.6499,  0.2861, -0.1187, -0.4263, -0.6296, -0.2539]])),
             ('2.bias', tensor([0.]))])

#### 自定义初始化

In [39]:
def my_init(m):
    if type(m) == nn.Linear:
        print("初始化参数：", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.abs() <= 5

In [40]:
net.apply(my_init)

初始化参数： weight torch.Size([8, 4])
初始化参数： weight torch.Size([1, 8])


Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [41]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[ 0.0000,  0.0000,  0.0000,  3.7068],
                      [ 2.7179, -0.0000, -0.0000,  1.3512],
                      [ 0.0000, -0.9948, -0.0000,  2.2204],
                      [ 0.0000, -1.3212,  1.7576,  0.0000],
                      [ 0.0000,  1.5075, -0.1535, -0.0000],
                      [ 1.9311,  0.0000,  0.0000, -1.6526],
                      [ 4.3464, -0.0000,  0.6213,  0.0000],
                      [-2.4103, -0.1730,  2.4200,  3.9227]])),
             ('0.bias', tensor([0., 0., 0., 0., 0., 0., 0., 0.])),
             ('2.weight',
              tensor([[-0.0000, -0.1962,  4.9190,  2.3955, -1.3821, -4.7296, -0.0000,  0.0000]])),
             ('2.bias', tensor([0.]))])

In [47]:
net[2].weight.data[:] -= 1

In [50]:
net[2].weight.data[0, 1] = 2099

In [51]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.0000e+00,  2.0990e+03,  4.9190e+00,  2.3955e+00, -1.3821e+00,
                       -4.7296e+00,  0.0000e+00,  0.0000e+00]])),
             ('bias', tensor([0.]))])

#### 3. 参数绑定

In [52]:
shared = nn.Linear(8, 8)  # 共享层

In [53]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.Linear(8,1 ))

In [57]:
net[2].weight.data == net[4].weight.data

tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])